# 🧩📊 Handling Missing Data in Nutrition Science

We’ll use a large cohort (n≈25,000; age 45–80) to explore missingness and compare four approaches on the same target model:

**Outcome**: `BMI_Baseline`

**Predictors**: Age, Sex, Smoking, Physical_Activity, Social_Class (UK A/B/C1/C2/D/E), Sugar_Intake, SFA_Intake

Methods compared:
1) Listwise deletion
2) Mean/Mode imputation
3) Multiple Imputation (m=5) with Rubin’s rules
4) Bayesian joint model (numeric missings latent; weakly-informative priors)

The notebook loads `data/epidemiological_study.csv` **or simulates** a realistic dataset (seed=11088).

## Environment
Uncomment installs if needed.

In [ ]:
# %pip install pandas numpy matplotlib seaborn scipy scikit-learn statsmodels pymc arviz
from pathlib import Path
import numpy as np, pandas as pd
import matplotlib.pyplot as plt, seaborn as sns
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.experimental import enable_iterative_imputer  # noqa: F401
from sklearn.impute import IterativeImputer
import pymc as pm
import arviz as az

import pytensor.tensor as pt

sns.set_context('notebook'); sns.set_style('whitegrid')
RANDOM_SEED = 11088
rng = np.random.default_rng(RANDOM_SEED)
DATA_DIR = Path('data'); DATA_DIR.mkdir(exist_ok=True)
DATA_PATH = DATA_DIR / 'epidemiological_study.csv'

## Load or Simulate Dataset
If the CSV is missing, we simulate a cohort with ~5–10% MCAR per variable and plausible nutrition/CVD structure.

In [ ]:
def simulate_epidemiology(n=25_000, seed=RANDOM_SEED):
    rng = np.random.default_rng(seed)
    age = rng.normal(62, 8.5, n).clip(45, 85)
    sex = rng.choice(['Female','Male'], n)
    smoking = rng.choice(['Non-smoker','Smoker'], n, p=[0.7,0.3])
    activity = rng.choice(['Low','Medium','High'], n, p=[0.3,0.45,0.25])
    social = rng.choice(['A','B','C1','C2','D','E'], n, p=[0.08,0.15,0.22,0.22,0.2,0.13])
    sugar = rng.normal(90, 35, n).clip(10, 250)
    sfa = rng.normal(28, 10, n).clip(5, 80)
    x_male = (sex=='Male').astype(int)
    x_act = pd.Series(activity).map({'Low':0,'Medium':1,'High':2}).values
    bmi0 = (rng.normal(27, 3.5, n) + 0.08*(age-62) + 0.6*x_male - 0.5*x_act + 0.015*(sugar-90))
    bp0 = rng.normal(132, 14, n) + 0.12*(age-62) + 1.2*(bmi0-27)
    df = pd.DataFrame({
        'ID': np.arange(1, n+1), 'Age': age.round(1), 'Sex': sex, 'Smoking': smoking,
        'Physical_Activity': activity, 'Social_Class': social,
        'Sugar_Intake': sugar.round(1), 'SFA_Intake': sfa.round(1),
        'BMI_Baseline': bmi0.round(2), 'BP_Baseline': bp0.round(1)
    })
    miss_cols = ['Sugar_Intake','SFA_Intake','Physical_Activity','Social_Class','BMI_Baseline','BP_Baseline','Age']
    for c in miss_cols:
        m = rng.random(len(df)) < rng.uniform(0.05,0.10)
        df.loc[m, c] = np.nan
    return df

if DATA_PATH.exists():
    data = pd.read_csv(DATA_PATH)
else:
    data = simulate_epidemiology()
    data.to_csv(DATA_PATH, index=False)
    print(f"Simulated and saved to {DATA_PATH}")

data.head()

## Step 1 — Explore Missingness

In [ ]:
miss_pct = (data.isna().mean()*100).round(2).sort_values(ascending=False)
display(miss_pct[miss_pct>0].to_frame('Missing_%'))
plt.figure(figsize=(10,6));
sns.heatmap(data.sample(min(2000,len(data))).isna(), cbar=False)
plt.title('Missingness heatmap (sample rows)'); plt.xlabel('Variables'); plt.ylabel('Participants'); plt.tight_layout(); plt.show()

## Step 2 — Prepare Model Matrices
Target model (for all methods):

**BMI_Baseline ~ Age + Sex + Smoking + Physical_Activity + Social_Class + Sugar_Intake + SFA_Intake**

We avoid `LabelEncoder`; we’ll either use a formula with `C()` or one-hot dummies with references.

In [ ]:
cols = ['BMI_Baseline','Age','Sex','Smoking','Physical_Activity','Social_Class','Sugar_Intake','SFA_Intake']
dfm = data[cols].copy()
# set explicit category orders (stable references)
dfm['Physical_Activity'] = dfm['Physical_Activity'].astype('category').cat.set_categories(['Low','Medium','High'])
dfm['Social_Class'] = dfm['Social_Class'].astype('category').cat.set_categories(['A','B','C1','C2','D','E'])
dfm.head(3)

## Step 3 — Listwise Deletion
Drop rows with any missing in the model variables; fit OLS by formula (gives CIs).

In [ ]:
lw = dfm.dropna().copy()
formula = 'BMI_Baseline ~ Age + C(Sex) + C(Smoking) + C(Physical_Activity, Treatment(reference="Medium")) + C(Social_Class, Treatment(reference="C1")) + Sugar_Intake + SFA_Intake'
ols_lw = smf.ols(formula, data=lw).fit()
print(f"Rows kept: {len(lw)} / {len(dfm)}")
lw_tab = pd.concat([ols_lw.params, ols_lw.conf_int()], axis=1)
lw_tab.columns = ['coef','2.5%','97.5%']
lw_tab.round(3)

## Step 4 — Mean/Mode Imputation
Mean for numeric; mode for categoricals. Then OLS by formula.

In [ ]:
mm = dfm.copy()
for c in ['BMI_Baseline','Age','Sugar_Intake','SFA_Intake']:
    mm[c] = mm[c].fillna(mm[c].mean())
for c in ['Sex','Smoking','Physical_Activity','Social_Class']:
    mm[c] = mm[c].fillna(mm[c].mode().iloc[0])
ols_mm = smf.ols(formula, data=mm).fit()
mm_tab = pd.concat([ols_mm.params, ols_mm.conf_int()], axis=1)
mm_tab.columns = ['coef','2.5%','97.5%']
mm_tab.round(3)

## Step 5 — Multiple Imputation (m=5) with Rubin’s Rules
We use `IterativeImputer` with posterior sampling and different seeds to create m datasets, fit the **same OLS**, and pool estimates and variances.

In [ ]:
def impute_once(df, seed):
    # Build a numeric-friendly frame: model.matrix via patsy formula
    # Use get_dummies for predictors; keep outcome separate to avoid dummying it
    base = df.copy()
    # Build dummy design with explicit references matching the formula
    X_cat = pd.get_dummies(base[['Sex','Smoking','Physical_Activity','Social_Class']], drop_first=True)
    X_num = base[['Age','Sugar_Intake','SFA_Intake']]
    y = base[['BMI_Baseline']]
    Z = pd.concat([y, X_num, X_cat], axis=1)
    imp = IterativeImputer(random_state=seed, sample_posterior=True, max_iter=20)
    Zi = pd.DataFrame(imp.fit_transform(Z), columns=Z.columns)
    # reconstruct a data frame close to original for formula fit
    out = base.copy()
    out['BMI_Baseline'] = Zi['BMI_Baseline']
    out['Age'] = Zi['Age']; out['Sugar_Intake'] = Zi['Sugar_Intake']; out['SFA_Intake'] = Zi['SFA_Intake']
    # For categoricals, fallback to original with any remaining missings filled by mode (IterativeImputer worked on dummies only)
    for c in ['Sex','Smoking','Physical_Activity','Social_Class']:
        out[c] = out[c].fillna(out[c].mode().iloc[0])
    return out

def rubins_rules(estimates, covs):
    # estimates: list of params Series; covs: list of covariance DataFrames
    params = estimates[0].index
    m = len(estimates)
    Qbar = pd.concat(estimates, axis=1).mean(axis=1)
    Ubar = sum(covs)/m
    B = pd.concat([(e - Qbar).to_frame() for e in estimates], axis=1)
    B = (B.pow(2)).mean(axis=1)
    T = Ubar.values.diagonal() + (1 + 1/m) * B.values
    se = np.sqrt(T)
    pooled = pd.DataFrame({'coef': Qbar, 'SE': se}, index=params)
    pooled['2.5%'] = pooled['coef'] - 1.96*pooled['SE']
    pooled['97.5%'] = pooled['coef'] + 1.96*pooled['SE']
    return pooled

M = 5
ests, covs = [], []
for i in range(M):
    seed = RANDOM_SEED + i
    mi_df = impute_once(dfm, seed)
    fit = smf.ols(formula, data=mi_df).fit()
    ests.append(fit.params)
    covs.append(fit.cov_params())

mi_tab = rubins_rules(ests, covs).round(3)
mi_tab

## Step 6 — Bayesian Joint Model with Latent Missing Values
We fit a Bayesian linear model for BMI with **weakly informative priors**. For *numeric* predictors (`Age`, `Sugar_Intake`, `SFA_Intake`, and `BMI_Baseline` if needed), we treat missing entries as latent variables.

Categorical missings are imputed with mode for simplicity (students can extend to categorical models). Numeric predictors are standardised to help sampling geometry.

In [ ]:

# --- Design matrix (categoricals -> dummies; numeric kept for latent imputation) ---
bx = dfm.copy()
for c in ['Sex','Smoking','Physical_Activity','Social_Class']:
    bx[c] = bx[c].fillna(bx[c].mode().iloc[0])

X_dum = pd.get_dummies(
    bx[['Sex','Smoking','Physical_Activity','Social_Class']],
    drop_first=True
)

# Numeric arrays with NaNs preserved for latent imputation
age   = dfm['Age'].to_numpy(dtype=float)
sugar = dfm['Sugar_Intake'].to_numpy(dtype=float)
sfa   = dfm['SFA_Intake'].to_numpy(dtype=float)
y_bmi = dfm['BMI_Baseline'].to_numpy(dtype=float)

# Standardise with NaN support; guard zero variance
def z_with_nan(x):
    m = np.nanmean(x)
    s = np.nanstd(x)
    if not np.isfinite(s) or s == 0:
        s = 1.0
    return (x - m) / s, (m, s)

age_z,   age_stats   = z_with_nan(age)
sugar_z, sugar_stats = z_with_nan(sugar)
sfa_z,   sfa_stats   = z_with_nan(sfa)

X_fix   = X_dum.astype(float).to_numpy()
fix_names = X_dum.columns.tolist()
N = X_fix.shape[0]

coords = {"obs": np.arange(N), "fix": fix_names}

with pm.Model(coords=coords) as bim:
    # Fixed (non-imputed) design part for categoricals
    X_fix_data = pm.Data('X_fix', X_fix, dims=('obs','fix'))

    # Latent z-scores for missing numeric predictors (length N each)
    age_lat   = pm.Normal('Age_z',   mu=0, sigma=1, dims=('obs',))
    sugar_lat = pm.Normal('Sugar_z', mu=0, sigma=1, dims=('obs',))
    sfa_lat   = pm.Normal('SFA_z',   mu=0, sigma=1, dims=('obs',))

    # Use observed z-scores where available; otherwise latent
    age_obs   = pt.as_tensor_variable(age_z)
    sugar_obs = pt.as_tensor_variable(sugar_z)
    sfa_obs   = pt.as_tensor_variable(sfa_z)

    age_use   = pt.where(pt.isnan(age_obs),   age_lat,   age_obs)
    sugar_use = pt.where(pt.isnan(sugar_obs), sugar_lat, sugar_obs)
    sfa_use   = pt.where(pt.isnan(sfa_obs),   sfa_lat,   sfa_obs)

    # Priors
    beta0     = pm.Normal('Intercept', 27, 5)
    beta_age  = pm.Normal('b_age',   0, 1)
    beta_sug  = pm.Normal('b_sugar', 0, 1)
    beta_sfa  = pm.Normal('b_sfa',   0, 1)
    beta_fix  = pm.Normal('Beta_fix', 0, 1, dims=('fix',))
    sigma     = pm.HalfNormal('Sigma', 3)

    # Linear predictor
    mu = beta0 + beta_age*age_use + beta_sug*sugar_use + beta_sfa*sfa_use + pm.math.dot(X_fix_data, beta_fix)

    # Likelihood (allow missing y via masked array)
    y_masked = np.ma.masked_invalid(y_bmi)
    pm.Normal('y_obs', mu=mu, sigma=sigma, observed=y_masked)

    trace_bim = pm.sample(
        draws=1500, tune=1000, target_accept=0.9,
        random_seed=RANDOM_SEED, return_inferencedata=True
    )

print(az.summary(trace_bim, var_names=['Intercept','b_age','b_sugar','b_sfa','Beta_fix','Sigma'], hdi_prob=0.95))


## Step 7 — Compare Estimates Across Methods
We harmonise coefficients for shared terms across methods.

In [ ]:
# --- Frequentist tables (unchanged) ---
def tidy(res):
    out = pd.concat([res.params, res.conf_int()], axis=1)
    out.columns = ['coef','2.5%','97.5%']
    return out.rename(index={'const': 'Intercept'})

lw_tab2 = tidy(ols_lw)
mm_tab2 = tidy(ols_mm)
mi_tab2 = mi_tab[['coef','2.5%','97.5%']].rename(index={'const':'Intercept'})

# --- Bayesian table from raw samples (no ArviZ summary/HDI) ---
def et_ci(samples, q=(0.025, 0.975)):
    """Equal-tailed CI for 1D numpy samples."""
    lo, hi = np.quantile(samples, q)
    return float(lo), float(hi)

rows = []

# Scalars first
for var, label in [
    ('Intercept','Intercept'),
    ('b_age','Age'),
    ('b_sugar','Sugar_Intake'),
    ('b_sfa','SFA_Intake'),
]:
    s = trace_bim.posterior[var].values.reshape(-1)   # [chains*draws]
    m = float(np.mean(s))
    lo, hi = et_ci(s)
    rows.append((label, m, lo, hi))

# Vector of dummy/fixed effects: handle either 'Beta_fix' or 'Beta'
beta_var = 'Beta_fix' if 'Beta_fix' in trace_bim.posterior.data_vars else 'Beta'
beta_da  = trace_bim.posterior[beta_var]                       # dims: chain, draw, <coefdim>
coefdim  = [d for d in beta_da.dims if d not in ('chain','draw')][0]
coef_idx = beta_da[coefdim].values.tolist()                    # names

# stack chains/draws -> samples per coefficient
beta_samples = (beta_da
                .stack(sample=('chain','draw'))                # dims: <coefdim>, sample
                .transpose(coefdim, 'sample')
                .values)                                       # shape: [n_coef, n_samples]

beta_mean = beta_samples.mean(axis=1)
beta_lo, beta_hi = np.quantile(beta_samples, [0.025, 0.975], axis=1)

for name, m, lo, hi in zip(coef_idx, beta_mean, beta_lo, beta_hi):
    rows.append((name, float(m), float(lo), float(hi)))

bayes_tab = pd.DataFrame(rows, columns=['term','coef','2.5%','97.5%']).set_index('term')

# --- Align and compare ---
common = (lw_tab2.index
          .intersection(mm_tab2.index)
          .intersection(mi_tab2.index)
          .intersection(bayes_tab.index))

cmp = pd.DataFrame({
    'Listwise':  lw_tab2.loc[common, 'coef'],
    'Mean/Mode': mm_tab2.loc[common, 'coef'],
    'MI (m=5)':  mi_tab2.loc[common, 'coef'],
    'Bayesian':  bayes_tab.loc[common, 'coef'],
}).sort_index()

cmp.round(3)

In [ ]:
plt.figure(figsize=(12,6))
cmp.plot(kind='bar', figsize=(12,6))
plt.title('Coefficient comparison across methods')
plt.ylabel('Estimate')
plt.xticks(rotation=45, ha='right')
plt.tight_layout(); plt.show()

## Learning Points
- **Listwise deletion**: simple, but reduces n and can bias if not MCAR.
- **Mean/Mode**: preserves n but underestimates uncertainty and shrinks relationships.
- **Multiple Imputation**: reflects imputation uncertainty; pooled CIs via Rubin’s rules.
- **Bayesian joint**: treats missings as unknowns; returns posterior for model parameters and latent values.

### Extensions for students
- Replace mean/mode for categoricals with multinomial imputation.
- Add **interactions** and **non-linearities** (splines) to the BMI model.
- Use diagnostics (e.g., posterior predictive checks) to assess imputation plausibility.
- Compare MI with different `m` and different imputation models.